## Comments

### Requirements for the data:
* Include prompts in 'prompt' column.
* Data have empty 'date' and 'Output' columns.
* Include an env. file with 'ENDPOINT', 'KEY', 'DEPLOYMENT_NAME' and 'MODEL_PREVIE' variables from recent a deployment model.


## Libraries & Set up Azure OpenAI

In [ ]:
import os
import openai
import numpy
import matplotlib.pyplot as plt
import pandas as pd
from openai import AzureOpenAI
import openpyxl
import datetime
import xlsxwriter

In [ ]:
endpoint = os.getenv("ENDPOINT", default=None)
key = os.getenv("KEY", default=None)
deployment_name = os.getenv("DEPLOYMENT_NAME", default=None)
model_preview = os.getenv("MODEL_PREVIEW", default=None)

In [ ]:
from dotenv import load_dotenv
load_dotenv()
client = AzureOpenAI(
    api_key=key,
    api_version=model_preview,
    azure_endpoint = endpoint
    )
model=deployment_name

## Read prompts data

In [ ]:
file_path = 'data/prompts.xlsx'
save_filename = "data/output.xlsx"

In [ ]:
def read_excel_file(file_path):
    df_list = []
    xls = pd.ExcelFile(file_path)
    workbook = openpyxl.load_workbook(file_path)
    page_names = workbook.sheetnames
    for i in page_names:
        sheet_df = pd.read_excel(xls, i)
        df_list.append(sheet_df)
    return page_names, df_list

## Get outputs

In [ ]:
def get_completion(model, prompt):
    prompt = prompt
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    output = response.choices[0].message.content
    return output

In [ ]:
def get_output_column(df):
    for row in df.itertuples():
        prompt = str(row.prompt)
        print(prompt)
        output = get_completion(model, prompt).lstrip()
        print(output)
        df.at[row.Index, "Output"] = output
    return df

In [ ]:
def time_column(df):
    current_date = datetime.date.today()
    for row in df.itertuples():
        df.at[row.Index, "date"] = current_date
    return df

### To get output for a specific page in excel

In [ ]:
def get_index_by_name(name, list):
    try:
        return list.index(name)
    except ValueError:
        return -1

In [ ]:
def get_dataframe_from_a_page(page_names, df, page_name):
    index= get_index_by_name(page_name, page_names)
    df = df[index]
    return df

In [ ]:
def save_df(df, output_file_name, page_name):
    writer = pd.ExcelWriter(output_file_name, engine='xlsxwriter')
    sheet_name = page_name
    df.to_excel(writer, sheet_name=sheet_name, index=False)
    writer.close()

In [ ]:
def get_output_dataset(page_names, df, output_file_name, page_name):
    df = get_dataframe_from_a_page(page_names, df, page_name)
    df= get_output_column(df)
    df = time_column(df)
    save_df(df, output_file_name, page_name)
    return df

### To obtain outputs for all pages in excel

In [ ]:
def get_output_dataset_for_list(file_path):
    list_df_with_outputs = []
    pages_names, df_list = read_excel_file(file_path)
    for i in df_list:
        print("i in the second", i)
        df= get_output_column(i)
        df = time_column(i)
        list_df_with_outputs.append(df)
    return list_df_with_outputs

In [ ]:
def save_dfs_to_excel(file_path, output_file_name):
    pages_names, df_list = read_excel_file(file_path)
    list_df_with_outputs = get_output_dataset_for_list(file_path)
    with pd.ExcelWriter(output_file_name, engine='xlsxwriter') as writer:
        for df, name in zip(list_df_with_outputs, pages_names):
            df.to_excel(writer, sheet_name=name, index=False)
    writer.close()

### Completions

In [ ]:
page_names, df = read_excel_file(file_path2)
df

In [ ]:
page_names

In [ ]:
save_dfs_to_excel(file_path, output_file_name)